In [98]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/manthanfursule/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [99]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/manthanfursule/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [100]:
nltk.download('stopwords')
stopwords_english = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/manthanfursule/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [101]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer  
import string
import contractions
import emoji
from nltk.stem import WordNetLemmatizer 
from datetime import datetime
import emot
from nltk.corpus import wordnet

In [102]:
lemmatizer = WordNetLemmatizer()

In [103]:
#stemmer = PorterStemmer() 

In [104]:
tokenizer = TweetTokenizer(preserve_case=False, 
                           strip_handles=True,
                           reduce_len=True)

In [105]:
emot_obj = emot.core.emot()

In [106]:
data = pd.read_csv("Twitter/1Mayo_ENGLISH_tweets_stweet.csv")

In [107]:
data.shape

(4502, 24)

In [108]:
'''
data = data.drop(['id_str', 'conversation_id_str', 'lang', 'favorite_count',
       'reply_count', 'quote_count', 'quoted_status_id_str',
       'quoted_status_short_url', 'quoted_status_expand_url', 'user_id_str',
       'user_name', 'user_full_name', 'user_verified',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str',
       'mentions', 'urls', 'media'],axis = 1)
'''

"\ndata = data.drop(['id_str', 'conversation_id_str', 'lang', 'favorite_count',\n       'reply_count', 'quote_count', 'quoted_status_id_str',\n       'quoted_status_short_url', 'quoted_status_expand_url', 'user_id_str',\n       'user_name', 'user_full_name', 'user_verified',\n       'in_reply_to_status_id_str', 'in_reply_to_user_id_str',\n       'mentions', 'urls', 'media'],axis = 1)\n"

In [109]:
#data = data.dropna(subset = ['full_text'])

In [110]:
#data.isnull().sum()*100/len(data)

In [111]:
data_copy = data.copy()

In [112]:
data_copy.head()

,created_at,id_str,conversation_id_str,full_text,lang,favorited,retweeted,retweet_count,favorite_count,reply_count,...,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media
0,2021-04-23T07:14:36+00:00,1385492282971987968,1385492282971987968,📢Join the seminar aiming at supporting #1May #...,en,False,False,5,4,0,...,1054637281389223936,_EuropeanForum,European Forum,False,NaN,NaN,"#1May , #1maggio , #1Mayo","LeftHeinz , Yolanda_Diaz_ , VisentiniLuca , lu...",NaN,"[{""url"": ""https://pbs.twimg.com/media/EzpAFBfX..."
1,2021-04-22T04:12:43+00:00,1385084126307528709,1384884688829943809,@julioacosta1701 @DiazCanelB @AlejandroGilF @r...,es,False,False,7,1,0,...,1210365797702492160,BauzaCuba,Alejandro Rodríguez Bauza,False,1.384885e+18,1.194288e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","julioacosta1701 , DiazCanelB , AlejandroGilF ,...",NaN,[]
2,2021-04-22T04:12:00+00:00,1385083944001998850,1384803265381031938,@julioacosta1701 @AliRubioGlez @Vicente7397772...,es,False,False,7,1,0,...,1210365797702492160,BauzaCuba,Alejandro Rodríguez Bauza,False,1.384889e+18,1.194288e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","julioacosta1701 , AliRubioGlez , Vicente739777...",NaN,[]
3,2021-04-22T04:09:48+00:00,1385083390022586370,1384882418755178498,@AdrinMartnezGo1 @julioacosta1701 @DiazCanelB ...,es,False,False,6,0,0,...,1210365797702492160,BauzaCuba,Alejandro Rodríguez Bauza,False,1.385057e+18,1.136331e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","AdrinMartnezGo1 , julioacosta1701 , DiazCanelB...",NaN,[]
4,2021-04-22T03:32:26+00:00,1385073986363658244,1384941470390661126,@VilmaPrez14 @Laura53538740 @Granma_Digital @D...,es,False,False,5,1,0,...,1210365797702492160,BauzaCuba,Alejandro Rodríguez Bauza,False,1.384977e+18,1.101197e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","VilmaPrez14 , Laura53538740 , Granma_Digital ,...",NaN,[]


In [113]:
emoji_pattern = re.compile("["
                           u"\U00002728-\U0001FAD6"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
Emoji_list = []
for line in data['full_text']:
    temp = []
    m = re.search(emoji_pattern,line)   
    if m is not None:
        temp.append(line[m.start():m.end()])
    Emoji_list.append((temp))

In [114]:
preprocessed_dates = []
for i in range(len(data_copy)):
    preprocessed_dates.append(datetime.strptime(data_copy['created_at'].values[i].split('T')[0], "%Y-%m-%d").date())

data['preprocessed_created_at'] = preprocessed_dates

In [115]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [116]:
count=0
url_pattern = r'https?://\S+|www\.\S+'
preprocessed_tweets_list = []
hash_pattern = r'#'
mention_pattern = r'@\S+'
digits = r'[0-9]'
emoticon_list = []
punctuation_pattern = r'[^\w\s]'
for i in range(len(data_copy)):
    contraction_removed_tweet = []
    preprocessed_tweets = []
    data_copy['full_text'].values[i] = data['full_text'].values[i].lower()
    data_copy['full_text'].values[i] = data_copy['full_text'].values[i].replace("\n"," ")
    data_copy['full_text'].values[i] = re.sub(pattern=url_pattern, repl=' ', string=data_copy['full_text'].values[i])
    data_copy['full_text'].values[i] = re.sub(pattern=hash_pattern, repl=' ', string=data_copy['full_text'].values[i])
    data_copy['full_text'].values[i] = re.sub(pattern=mention_pattern, repl=' ', string=data_copy['full_text'].values[i])
    emoticon_list.append(emot_obj.emoticons(data_copy['full_text'].values[i])['value'])
    data_copy['full_text'].values[i] = re.sub(pattern=digits, repl=' ', string=data_copy['full_text'].values[i])
    data_copy['full_text'].values[i] = re.sub(pattern=emoji_pattern, repl='', string=data_copy['full_text'].values[i])
    data_copy['full_text'].values[i] = re.sub(pattern=punctuation_pattern, repl='', string=data_copy['full_text'].values[i])
    data_copy['full_text'].values[i] = tweet_tokens = tokenizer.tokenize(data_copy['full_text'].values[i])
    for word in data_copy['full_text'].values[i]:
        contraction_removed_tweet.append(contractions.fix(word))
    for word in contraction_removed_tweet:
        if word not in stopwords_english:  
            preprocessed_tweets.append(lemmatizer.lemmatize(word,get_wordnet_pos(word)))
    preprocessed_tweets_list.append(preprocessed_tweets)
data['preprocessed_data'] = preprocessed_tweets_list
data['emoji_list'] = Emoji_list
data['emoticon_list'] = emoticon_list

In [117]:
print(data["preprocessed_data"][0])

['join', 'seminar', 'aim', 'support', 'may', 'maggio', 'mayo', 'trade', 'union', 'activity', 'amp', 'mobilization']


In [118]:
data.columns

Index(['created_at', 'id_str', 'conversation_id_str', 'full_text', 'lang',
       'favorited', 'retweeted', 'retweet_count', 'favorite_count',
       'reply_count', 'quote_count', 'quoted_status_id_str',
       'quoted_status_short_url', 'quoted_status_expand_url', 'user_id_str',
       'user_name', 'user_full_name', 'user_verified',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'hashtags',
       'mentions', 'urls', 'media', 'preprocessed_created_at',
       'preprocessed_data', 'emoji_list', 'emoticon_list'],
      dtype='object')